# Feb 15, 2025: visualize membership matrices 
on brain, per solution mode, per animal

In [ ]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob
import random

# import arviz as az

import ants
from nipype.interfaces import afni

from itertools import product, combinations, chain
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.source ='allen' #'spatial' #'allen'
args.space = 'ccfv2' #'yale' #'ccfv2'
args.brain_div = 'whl' #'whl' #'whl'
args.num_rois = 172 #162 #172
args.resolution = 200 #200 #200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-200'

In [4]:
args.high_res = 25

PARC_DESC_HR = ( # High Resolution
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.high_res}'
)
PARC_DESC_HR

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-25'

In [5]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.THRESHOLD = f'signed'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'sub'

BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = (
    f'{BASE_path}/roi-results-v3'
    f'/{PARC_DESC}'
)
TS_path = f'{ROI_path}/roi_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLD}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
DIAG_path = f'{ROI_RESULTS_path}/diagnostics'
os.system(f'mkdir -p {DIAG_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [6]:
parcels_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')

try:
    roi_labels = pd.read_csv(f'{PARCELS_path}/{PARC_DESC}_desc-names.csv')['roi'].to_list()
except:
    roi_labels = np.arange(1, args.num_rois+1)

In [7]:
# parcels_hr_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC_HR}_desc-parcels.nii.gz')

# roi_labels = pd.read_csv(f'{PARCELS_path}/{PARC_DESC_HR}_desc-names.csv')['roi'].to_list()

In [8]:
parcels_img

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (57, 40, 66)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (5.78, 7.83, -6.93)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

In [9]:
def concatenate(in_files, out_file):
    try:
        os.remove(out_file)
    except:
        pass

    tcat = afni.TCat()
    tcat.inputs.in_files = in_files
    tcat.inputs.out_file = out_file
    tcat.inputs.rlt = ''
    tcat.cmdline 
    tcat.run()

    for file in in_files:
        try:
            os.remove(file)
        except:
            pass
    return None

def marginal_to_nifti(args, parcels_img, X, mode_id, folder, level=-1):
    parcels_vol = parcels_img.numpy()
    parcels_vol = np.round(parcels_vol, decimals=0)

    os.system(f'mkdir -p {folder}')
    args.num_rois, args.num_comms = X.shape

    in_files = []
    for idx_comm in range(args.num_comms):
        x = X[:, idx_comm]
        x_img = np.zeros_like(parcels_vol)
        for idx, roi in enumerate(roi_labels):
            x_img += (parcels_vol == roi) * (x[idx])
        
        file = f'{folder}/mode-{mode_id}_comm-{idx_comm}.nii.gz'
        parcels_img.new_image_like(x_img).to_filename(file)
        in_files.append(file)

    if level == -1:
        out_file = f'{folder}/mode-{mode_id:02d}.nii.gz'
    else:
        out_file = f'{folder}/mode-{mode_id:02d}_level-{level}.nii.gz'
    concatenate(in_files, out_file)
    return None

In [10]:
args.dc, args.sbm = False, 'h'

args.nested = True if args.sbm in ['h'] else False

args.force_niter = 40000
args.num_draws = int((1/2) * args.force_niter)

def sbm_name(args):
    dc = f'dc' if args.dc else f'nd'
    dc = f'' if args.sbm in ['m', 'a'] else dc
    file = f'sbm-{dc}-{args.sbm}'
    return file

SBM = sbm_name(args)
SBM

'sbm-nd-h'

In [11]:
def get_membership_matrix(num_rois, df, col='pi'):
    pis = [np.zeros((num_rois, 1)) if np.isnan(pi).all() else pi for pi in df[col]]

    num_modes = len(df)
    num_comms = np.max([pi.shape[-1] for pi in pis])
    num_rois = num_rois
    M = np.zeros((num_rois, num_modes, num_comms)) # membership profile matrix

    for idx_mode, pi in enumerate(pis):
        M[:, idx_mode, :pi.shape[-1]] = pi
    
    return M

In [12]:
marginals_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/partition-modes-group-aligned/{SBM}/desc-mem-mats.pkl', recursive=True))
marginals_df = []
for sbm_file in marginals_files:
    with open(f'{sbm_file}', 'rb') as f:
        row = pickle.load(f)
    marginals_df += [row]
marginals_df = pd.concat(marginals_df).reset_index(drop=True)
mode_ids = list(chain.from_iterable([list(range(count)) for count in marginals_df['sub'].value_counts().sort_index().to_list()]))
marginals_df['mode_id'] = mode_ids
marginals_df

,sub,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,pi_8_aligned,omega,mode_id
0,SLC01,sbm-nd-h,"[[0.9943502824858758, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9082639088384565, 0.005617798206134892, 0....","[[0.8850912125784436, 0.09756830776993501, 0.0...","[[0.9881457547053836, 0.011854245294616511], [...","[[0.9952583018821535, 0.004741698117846605], [...","[[0.9976291509410767, 0.0023708490589233024], ...","[[0.9976291509410767, 0.0023708490589233024], ...","[[0.9976291509410767, 0.0023708490589233024], ...","[[1.0], [1.0], [1.0000000000000002], [1.0], [0...",0.177295,0
1,SLC01,sbm-nd-h,"[[0.9924242424242424, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9305555555555556, 0.0, 0.00751836547291092...","[[0.8583704854173055, 0.11905640763280642, 0.0...","[[0.9686317578785476, 0.03136824212145242], [0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...",0.132086,1
2,SLC01,sbm-nd-h,"[[0.9672131147540983, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9002956194571352, 0.0, 0.00792797635044343...","[[0.8373335842458809, 0.11451218593710696, 0.0...","[[0.9879460856908309, 0.012053914309169156], [...","[[1.0], [0.9999999999999999], [0.9999999999999...","[[1.0], [0.9999999999999999], [0.9999999999999...","[[1.0], [0.9999999999999999], [0.9999999999999...","[[1.0], [0.9999999999999999], [0.9999999999999...","[[1.0], [0.9999999999999999], [0.9999999999999...",0.122464,2
3,SLC01,sbm-nd-h,"[[0.989247311827957, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.9813851312290439, 0.0, 0.00011562030292519...","[[0.9137888107775382, 0.06490505861861601, 0.0...","[[0.9885271006993847, 0.011472899300615397], [...","[[0.997705420139877, 0.0022945798601230792], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",0.093199,3
4,SLC01,sbm-nd-h,"[[0.44565217391304346, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.9117334697438667, 0.028759166164648204, 0....","[[0.9033271327908874, 0.095322184980498, 0.000...","[[0.9909216966685239, 0.009078303331476], [0.9...","[[0.995460848334262, 0.004539151665738], [0.99...","[[0.995460848334262, 0.004539151665738], [0.99...","[[1.0], [1.0000000000000002], [1.0], [1.000000...","[[1.0], [1.0000000000000002], [1.0], [1.000000...","[[1.0], [1.0000000000000002], [1.0], [1.000000...",0.091858,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,SLC10,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.02105398883751773, 0.00040523863029342963,...","[[0.9629751425533718, 0.0367510004382298, 0.00...","[[0.9914532557120393, 0.008546744287960418], [...","[[0.9982906511424077, 0.0017093488575920836], ...","[[0.9999999999999998], [1.0], [1.0], [1.0], [1...","[[0.9999999999999998], [1.0], [1.0], [1.0], [1...","[[0.9999999999999998], [1.0], [1.0], [1.0], [1...","[[0.9999999999999998], [1.0], [1.0], [1.0], [1...",0.233420,1
81,SLC10,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.9869606077218264,...","[[0.9629514024881322, 0.037048597511867884, 0....","[[0.9919459570626374, 0.008054042937362584], [...","[[0.9991945957062638, 0.0008054042937362584], ...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...","[[1.0], [1.0], [1.0], [1.0], [1.0], [0.9999999...",0.226136,2
82,SLC10,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.014634146341463415, 0.0, 0.0, 0.0, 0.0, 0....","[[0.9943370542744164, 0.0054487874864401434, 0...","[[0.9995046556830509, 0.000495344316949104], [...","[[0.9997523278415255, 0.000247672158474552], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",

In [13]:
cols = [col for col in  list(marginals_df.columns) if 'pi_' in col]
cols

['pi_0_aligned',
 'pi_1_aligned',
 'pi_2_aligned',
 'pi_3_aligned',
 'pi_4_aligned',
 'pi_5_aligned',
 'pi_6_aligned',
 'pi_7_aligned',
 'pi_8_aligned']

In [14]:
# SOFT MARGINALS
soft_marginals_df = []
for sub, group in marginals_df.groupby('sub'):
    omegas = group['omega'].to_list()
    dct = {'sub': [sub], 'sbm': [SBM]}
    for col in cols:
        M = get_membership_matrix(args.num_rois, group, col=col)
        SCs = np.average(M, axis=1, weights=omegas) # soft-comms.
        SCs = np.round(SCs, decimals=3)
        dct[col] = [SCs]
    soft_marginals_df += [pd.DataFrame(dct)]
soft_marginals_df = pd.concat(soft_marginals_df).reset_index(drop=True)
soft_marginals_df

,sub,sbm,pi_0_aligned,pi_1_aligned,pi_2_aligned,pi_3_aligned,pi_4_aligned,pi_5_aligned,pi_6_aligned,pi_7_aligned,pi_8_aligned
0,SLC01,sbm-nd-h,"[[0.835, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.923, 0.008, 0.005, 0.005, 0.0, 0.005, 0.00...","[[0.891, 0.088, 0.021, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.986, 0.014], [0.982, 0.018], [0.983, 0.017...","[[0.998, 0.002], [0.997, 0.003], [0.997, 0.003...","[[0.999, 0.001], [0.999, 0.001], [0.999, 0.001...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
1,SLC03,sbm-nd-h,"[[0.996, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002, 0.0, ...","[[0.677, 0.0, 0.0, 0.001, 0.004, 0.001, 0.317,...","[[0.615, 0.322, 0.05, 0.0, 0.004, 0.0, 0.009, ...","[[0.943, 0.056, 0.0, 0.0, 0.001, 0.0], [0.955,...","[[0.992, 0.008], [0.994, 0.006], [0.993, 0.007...","[[0.999, 0.001], [0.999, 0.001], [0.999, 0.001...","[[0.999, 0.001], [1.0, 0.0], [0.999, 0.001], [...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
2,SLC04,sbm-nd-h,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.058, 0.222, 0.001, 0.207, 0.0, 0.501, 0.00...","[[0.84, 0.153, 0.005, 0.003, 0.0, 0.0, 0.0], [...","[[0.979, 0.021], [0.977, 0.023], [0.978, 0.022...","[[0.997, 0.003], [0.997, 0.003], [0.997, 0.003...","[[0.999, 0.001], [0.999, 0.001], [0.999, 0.001...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
3,SLC05,sbm-nd-h,"[[0.0, 0.0, 0.989, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.029, 0.087, 0.0, 0.882, 0.0, 0.0, 0.001, 0...","[[0.878, 0.115, 0.006, 0.0, 0.0, 0.0, 0.0], [0...","[[0.98, 0.02, 0.0, 0.0, 0.0], [0.98, 0.02, 0.0...","[[0.997, 0.003], [0.997, 0.003], [0.997, 0.003...","[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
4,SLC06,sbm-nd-h,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.002, 0.001, 0.007, 0.0, 0.0, 0.006, 0.0, 0...","[[0.909, 0.068, 0.015, 0.0, 0.0, 0.0, 0.008], ...","[[0.985, 0.015, 0.0, 0.0, 0.0, 0.0], [0.969, 0...","[[0.998, 0.002], [0.996, 0.004], [0.997, 0.003...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
5,SLC07,sbm-nd-h,"[[0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.014, 0.017, 0.0, 0.968, 0.0, 0.0, 0.0, 0.0...","[[0.85, 0.149, 0.001, 0.0, 0.0, 0.0, 0.0], [0....","[[0.98, 0.02, 0.0, 0.0, 0.0, 0.0], [0.981, 0.0...","[[0.995, 0.005], [0.995, 0.005], [0.995, 0.005...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
6,SLC08,sbm-nd-h,"[[0.047, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.06, 0.014, 0.0, 0.918, 0.0, 0.0, 0.005, 0....","[[0.882, 0.091, 0.026, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.983, 0.017, 0.0, 0.0, 0.0], [0.982, 0.018,...","[[0.998, 0.002], [0.998, 0.002], [0.998, 0.002...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
7,SLC09,sbm-nd-h,"[[0.008, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.052, 0.0, 0.002, 0.0, 0.024, 0.059, 0.036,...","[[0.926, 0.065, 0.005, 0.0, 0.0, 0.0, 0.003, 0...","[[0.99, 0.01, 0.0, 0.0, 0.0], [0.972, 0.026, 0...","[[0.999, 0.001], [0.998, 0.002], [0.998, 0.002...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1

In [15]:
for idx, row in marginals_df.iterrows():
    for col in cols[:]:
        pi = row[col]
        if np.any(np.isnan(pi)): continue
        mode_id = row['mode_id']
        sub = row['sub']
        level = col.split('_')[1]
        if level == 'aligned': level = -1

        # data resolution
        folder = (
            f'{ESTIM_path}/individual/sub-{sub}/partition-modes-group-aligned'
            f'/{SBM}/marginal-visuals/res-{args.resolution}/nii'
        )
        print(sub, mode_id)
        marginal_to_nifti(args, parcels_img, pi, mode_id, folder, level=level)

        # # higher resolution
        # folder = (
        #     f'{ESTIM_path}/individual/sub-{sub}/partition-modes-group-aligned'
        #     f'/{SBM}/marginal-visuals/res-{args.high_res}/nii'
        # )
        # marginal_to_nifti(args, parcels_hr_img, pi, mode_id, folder, level=level)

SLC01 0
250515-12:21:29,892 nipype.interface INFO:
	 stderr 2025-05-15T12:21:29.892313:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250515-12:21:29,913 nipype.interface INFO:
	 stderr 2025-05-15T12:21:29.913155:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
250515-12:21:29,916 nipype.interface INFO:
	 stderr 2025-05-15T12:21:29.916355:*+ WARNING: Set TR of output dataset to 1.0 s
250515-12:21:30,50 nipype.interface INFO:
	 stderr 2025-05-15T12:21:30.050663:++ elapsed time = 0.2 s
SLC01 0
250515-12:21:30,835 nipype.interface INFO:
	 stderr 2025-05-15T12:21:30.835701:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250515-12:21:30,844 nipype.interface INFO:
	 stderr 2025-05-15T12:21:30.844725:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
250515-12:21:30,847 nipype.interface INFO:
	 stderr 2025-05-15T12:21:30.847485:*+ WARNING: Set TR of output dataset to 1.0 s
250515-12:21:30,921 nipype.interface INFO:
	

In [16]:
for col in cols[:2]:
    M = get_membership_matrix(args.num_rois, soft_marginals_df, col=col)
    grp_pi = np.median(M, axis=1)
    
    level = col.split('_')[1]
    if level == 'aligned': level = -1

    # data resolution
    folder = (
        f'{ESTIM_path}/group/membership-mats-group-aligned'
        f'/{SBM}/marginal-visuals/res-{args.resolution}/nii'
    ) 
    os.system(f'mkdir -p {folder}')
    marginal_to_nifti(args, parcels_img, grp_pi, 0, folder, level)

    # # higher resolution
    # folder = (
    #     f'{ESTIM_path}/group/membership-mats-group-aligned'
    #     f'/{SBM}/marginal-visuals/res-{args.high_res}/nii'
    # ) 
    # os.system(f'mkdir -p {folder}')
    # marginal_to_nifti(args, parcels_hr_img, grp_pi, 0, folder, level)
    # break


250515-12:28:58,696 nipype.interface INFO:
	 stderr 2025-05-15T12:28:58.696551:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250515-12:28:58,722 nipype.interface INFO:
	 stderr 2025-05-15T12:28:58.722805:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
250515-12:28:58,725 nipype.interface INFO:
	 stderr 2025-05-15T12:28:58.725145:*+ WARNING: Set TR of output dataset to 1.0 s
250515-12:28:58,919 nipype.interface INFO:
	 stderr 2025-05-15T12:28:58.919871:++ elapsed time = 0.2 s
250515-12:29:00,115 nipype.interface INFO:
	 stderr 2025-05-15T12:29:00.115165:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
250515-12:29:00,130 nipype.interface INFO:
	 stderr 2025-05-15T12:29:00.129998:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
250515-12:29:00,132 nipype.interface INFO:
	 stderr 2025-05-15T12:29:00.132697:*+ WARNING: Set TR of output dataset to 1.0 s
250515-12:29:00,245 nipype.interface INFO:
	 stderr 2025-05

In [17]:
col.split('_')

['pi', '1', 'aligned']